# Stock Options Analysis

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import yfinance as yf
sns.set_style("whitegrid")
pd.set_option('display.max_columns', None)  # Set to display all columns of dataframes

## Data

Let us try to study the Black-Scholes Model, which require the information

i. ***Stock Price (S)***: the corresponding stock prices for each option.\

ii.***Strike Price (K)***: data under the column strike_price.

iii. ***Time to Expiry (T)***: subtracting the current date from the expiration date (exdate). Convert the resulting timedelta to years.

iv. ***Risk-free Interest Rate (r)***: determine an appropriate risk-free interest rate to use in the model. 

v. ***Volatility (σ)***: implied volatility, available in your data. 

### Load options data and download stock data

In [9]:
current_dir = os.getcwd()
# print("Current directory:", current_dir)

# Read the CSV file into a DataFrame
options_data = pd.read_csv('../data/options_data.csv')

print(options_data.info())


/var/folders/q5/69ygwl597nd8rn5pmqr01b580000gn/T/ipykernel_12342/4023190986.py:5: DtypeWarning: Columns (23,26,32) have mixed types. Specify dtype option on import or set low_memory=False.
  options_data = pd.read_csv('../data/options_data.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1386118 entries, 0 to 1386117
Data columns (total 38 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   secid             1386118 non-null  int64  
 1   date              1386118 non-null  object 
 2   symbol            1386118 non-null  object 
 3   symbol_flag       1386118 non-null  int64  
 4   exdate            1386118 non-null  object 
 5   last_date         1263749 non-null  object 
 6   cp_flag           1386118 non-null  object 
 7   strike_price      1386118 non-null  int64  
 8   best_bid          1386118 non-null  float64
 9   best_offer        1386118 non-null  float64
 10  volume            1386118 non-null  int64  
 11  open_interest     1386118 non-null  int64  
 12  impl_volatility   1166644 non-null  float64
 13  delta             1166644 non-null  float64
 14  gamma             1166644 non-null  float64
 15  vega              1166644 non-null  float64
 16  

## Clean data

In [10]:
options_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1386118 entries, 0 to 1386117
Data columns (total 38 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   secid             1386118 non-null  int64  
 1   date              1386118 non-null  object 
 2   symbol            1386118 non-null  object 
 3   symbol_flag       1386118 non-null  int64  
 4   exdate            1386118 non-null  object 
 5   last_date         1263749 non-null  object 
 6   cp_flag           1386118 non-null  object 
 7   strike_price      1386118 non-null  int64  
 8   best_bid          1386118 non-null  float64
 9   best_offer        1386118 non-null  float64
 10  volume            1386118 non-null  int64  
 11  open_interest     1386118 non-null  int64  
 12  impl_volatility   1166644 non-null  float64
 13  delta             1166644 non-null  float64
 14  gamma             1166644 non-null  float64
 15  vega              1166644 non-null  float64
 16  

In [11]:
options_data.isnull().sum()

secid                     0
date                      0
symbol                    0
symbol_flag               0
exdate                    0
last_date            122369
cp_flag                   0
strike_price              0
best_bid                  0
best_offer                0
volume                    0
open_interest             0
impl_volatility      219474
delta                219474
gamma                219474
vega                 219474
theta                219474
optionid                  0
cfadj                     0
am_settlement             0
contract_size             0
ss_flag                   0
forward_price       1386118
expiry_indicator     947992
root                1386118
suffix              1386118
cusip                     0
ticker                    0
sic                       0
index_flag                0
exchange_d                0
class               1386118
issue_type                0
industry_group      1351152
issuer                    0
div_convention      

#### Drop all columns with all missing values and unnecessary categorical columns 

After dropping columns that do not have values, we have the remainging columns as below. We can drop more columns as we understand the nature of dataset and those columns may not influence our machine learning model. 

secid: Security ID, a unique identifier for each security.

date: Date of the options data.

symbol: Symbol of the security.
 
symbol_flag: Flag indicating the type of symbol.
 
exdate: Expiration date of the option.

last_date: Last trading date of the option.

cp_flag: Call (C) or Put (P) option flag.

strike_price: Strike price of the option.

best_bid: Best bid price for the option.

best_offer: Best offer price for the option.

volume: Volume of contracts traded.

open_interest: Open interest for the option.

impl_volatility: Implied volatility of the option.

delta: Delta value of the option.

gamma: Gamma value of the option.

vega: Vega value of the option.

theta: Theta value of the option.

optionid: Option ID, a unique identifier for each option.

cfadj: Adjustment factor for cash flow.

am_settlement: American-style settlement flag.

contract_size: Size of the contract.

ss_flag: Special security flag.

expiry_indicator: Expiry indicator, 'w' meaning weekly

cusip: CUSIP number, a unique identifier for securities in the U.S. and Canada.

ticker: Ticker symbol of the security.

sic: Standard Industrial Classification (SIC) code.

index_flag: Index security flag.

exchange_d: Exchange identifier.

issue_type: Type of issue.

industry_group: Industry group.

issuer: Issuer of the security.

exercise_style: Exercise style of the option (e.g., American or European).

In [12]:
# Drop all columns with all missing values
options_data.dropna(axis=1, how='all', inplace=True)


In [13]:
options_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1386118 entries, 0 to 1386117
Data columns (total 32 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   secid             1386118 non-null  int64  
 1   date              1386118 non-null  object 
 2   symbol            1386118 non-null  object 
 3   symbol_flag       1386118 non-null  int64  
 4   exdate            1386118 non-null  object 
 5   last_date         1263749 non-null  object 
 6   cp_flag           1386118 non-null  object 
 7   strike_price      1386118 non-null  int64  
 8   best_bid          1386118 non-null  float64
 9   best_offer        1386118 non-null  float64
 10  volume            1386118 non-null  int64  
 11  open_interest     1386118 non-null  int64  
 12  impl_volatility   1166644 non-null  float64
 13  delta             1166644 non-null  float64
 14  gamma             1166644 non-null  float64
 15  vega              1166644 non-null  float64
 16  

In [14]:

# Drop some more categorical columns that are not necessary
columns_to_drop =['secid', 'symbol', 'symbol_flag', 'last_date', 'optionid',
                  'cfadj', 'am_settlement', 'ss_flag', 'expiry_indicator', 
                  'exchange_d', 'issue_type', 'industry_group', 'issuer',
                  'cusip', 'sic', 'index_flag', 'exercise_style'] 
options_data = options_data.drop(columns=columns_to_drop)

# Here are the remaining columns

#  0   date             1386118 non-null  object 
#  1   exdate           1386118 non-null  object 
#  2   cp_flag          1386118 non-null  object 
#  3   strike_price     1386118 non-null  int64  
#  4   best_bid         1386118 non-null  float64
#  5   best_offer       1386118 non-null  float64
#  6   volume           1386118 non-null  int64  
#  7   open_interest    1386118 non-null  int64  
#  8   impl_volatility  1166644 non-null  float64
#  9   delta            1166644 non-null  float64
#  10  gamma            1166644 non-null  float64
#  11  vega             1166644 non-null  float64
#  12  theta            1166644 non-null  float64
#  13  contract_size    1386118 non-null  int64  
#  14  ticker           1386118 non-null  object 


In [15]:
options_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1386118 entries, 0 to 1386117
Data columns (total 15 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   date             1386118 non-null  object 
 1   exdate           1386118 non-null  object 
 2   cp_flag          1386118 non-null  object 
 3   strike_price     1386118 non-null  int64  
 4   best_bid         1386118 non-null  float64
 5   best_offer       1386118 non-null  float64
 6   volume           1386118 non-null  int64  
 7   open_interest    1386118 non-null  int64  
 8   impl_volatility  1166644 non-null  float64
 9   delta            1166644 non-null  float64
 10  gamma            1166644 non-null  float64
 11  vega             1166644 non-null  float64
 12  theta            1166644 non-null  float64
 13  contract_size    1386118 non-null  int64  
 14  ticker           1386118 non-null  object 
dtypes: float64(7), int64(4), object(4)
memory usage: 158.6+ MB


#### Convert date columns to datetime

In [16]:
# Explore categorical columns
print("\nUnique values in cp_flag column:", options_data['cp_flag'].unique())
print("\nUnique values in ticker column:", options_data['ticker'].unique())
print("\nUnique values in contract_size column:", options_data['contract_size'].unique())

unique_open_interest = options_data['open_interest'].unique()
print("\nUnique values in open_interest column:", options_data['open_interest'].unique())

# for oi in unique_open_interest:
#     print(oi)



Unique values in cp_flag column: ['P' 'C']

Unique values in ticker column: ['AMD' 'NVDA' 'TSM' 'SMCI']

Unique values in contract_size column: [100 172]

Unique values in open_interest column: [   43     8   154 ... 42576 14955 15329]


***open_interest***
ref: https://www.investopedia.com/terms/o/openinterest.asp#:~:text=Open%20interest%20is%20the%20total,tracking%20the%20total%20volume%20traded.

"Open interest is the total number of outstanding derivative contracts for an asset—such as options or futures—that have not been settled. Open interest keeps track of every open position in a particular contract rather than tracking the total volume traded."

In [17]:
# Explore numerical columns
print("\nSummary statistics of numerical columns:\n", options_data.describe())


Summary statistics of numerical columns:
        strike_price      best_bid    best_offer        volume  open_interest  \
count  1.386118e+06  1.386118e+06  1.386118e+06  1.386118e+06   1.386118e+06   
mean   1.640545e+05  3.690964e+01  3.829978e+01  2.198346e+02   1.229064e+03   
std    1.096561e+05  5.998198e+01  6.119734e+01  1.725485e+03   3.150095e+03   
min    5.000000e+03  0.000000e+00  1.000000e-02  0.000000e+00   0.000000e+00   
25%    8.500000e+04  8.500000e-01  1.200000e+00  0.000000e+00   7.000000e+00   
50%    1.300000e+05  1.095000e+01  1.175000e+01  1.000000e+00   1.590000e+02   
75%    2.150000e+05  4.620000e+01  4.850000e+01  2.875000e+01   9.540000e+02   
max    6.000000e+05  4.853000e+02  4.915500e+02  1.764690e+05   9.757500e+04   

       impl_volatility         delta         gamma          vega  \
count     1.166644e+06  1.166644e+06  1.166644e+06  1.166644e+06   
mean      6.532276e-01 -1.093080e-02  9.271234e-03  1.646618e+01   
std       3.339506e-01  5.886337

In [18]:
# Double-check null values
options_data.isnull().sum()

# We observed that there are some missing values of the implied volatility and the Greeks. 

date                    0
exdate                  0
cp_flag                 0
strike_price            0
best_bid                0
best_offer              0
volume                  0
open_interest           0
impl_volatility    219474
delta              219474
gamma              219474
vega               219474
theta              219474
contract_size           0
ticker                  0
dtype: int64

#### Convert datatime

In [19]:
options_data['date'] = pd.to_datetime(options_data['date'])
options_data['exdate'] = pd.to_datetime(options_data['exdate'])


#### Check for dupplicates

In [20]:
options_data.drop_duplicates(inplace=True)

In [21]:
options_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1386118 entries, 0 to 1386117
Data columns (total 15 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   date             1386118 non-null  datetime64[ns]
 1   exdate           1386118 non-null  datetime64[ns]
 2   cp_flag          1386118 non-null  object        
 3   strike_price     1386118 non-null  int64         
 4   best_bid         1386118 non-null  float64       
 5   best_offer       1386118 non-null  float64       
 6   volume           1386118 non-null  int64         
 7   open_interest    1386118 non-null  int64         
 8   impl_volatility  1166644 non-null  float64       
 9   delta            1166644 non-null  float64       
 10  gamma            1166644 non-null  float64       
 11  vega             1166644 non-null  float64       
 12  theta            1166644 non-null  float64       
 13  contract_size    1386118 non-null  int64         
 14  ti

#### Check missing values

We observed that there are significant number of missing values of the columns : impl_volatility, and the Greeks. let us drop all these rows

In [27]:
options_data.isnull().sum()

date               0
exdate             0
cp_flag            0
strike_price       0
best_bid           0
best_offer         0
volume             0
open_interest      0
impl_volatility    0
delta              0
gamma              0
vega               0
theta              0
contract_size      0
ticker             0
dtype: int64

#### Implied Volatility and the Greeks
We need more investigation about the Greeks and the implied volatility, so let us just drop the missing values for now and study based on what we have. In the future, when we have better understanding about them, we may interpolate data instead of dropping them, which reduce the size ot the dataset significantly.

Ref: 
1.https://www.investopedia.com/terms/i/iv.asp#:~:text=Volatility%20(IV)%20Works-,Implied%20volatility%20is%20the%20market's%20forecast%20of%20a%20likely%20movement,the%20symbol%20%CF%83%20(sigma).

2. WQU

3. https://www.investopedia.com/trading/getting-to-know-the-greeks/#:~:text=Option%20Greeks%20are%20financial%20metrics,price%20of%20the%20underlying%20asset.


Implied Volatility: this is a significantly important regarding to stock options, which is the key to the options's price and also influence stock price. Other numeric inputs like, stock price, strike level, risk-free rate, and dividend yield, are easily observed in the market.


In [23]:
# Drop rows with missing values in the specified columns
options_data.dropna(subset=['impl_volatility', 'delta', 'gamma', 'vega', 'theta'], inplace=True)


In [ ]:
options_data.describe()

In [ ]:
options_data.head()

In [25]:
options_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1166644 entries, 0 to 1386117
Data columns (total 15 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   date             1166644 non-null  datetime64[ns]
 1   exdate           1166644 non-null  datetime64[ns]
 2   cp_flag          1166644 non-null  object        
 3   strike_price     1166644 non-null  int64         
 4   best_bid         1166644 non-null  float64       
 5   best_offer       1166644 non-null  float64       
 6   volume           1166644 non-null  int64         
 7   open_interest    1166644 non-null  int64         
 8   impl_volatility  1166644 non-null  float64       
 9   delta            1166644 non-null  float64       
 10  gamma            1166644 non-null  float64       
 11  vega             1166644 non-null  float64       
 12  theta            1166644 non-null  float64       
 13  contract_size    1166644 non-null  int64         
 14  ticker 

#### Encoding categorical data

Remember to encode categorical data

In [ ]:
data = options_data.copy()

# Perform one-hot encoding for cp_flag and ticker columns
encoded_data = pd.get_dummies(data, columns=['cp_flag', 'ticker'], drop_first=False)


In [ ]:
encoded_data.head()

In [ ]:
encoded_data.info()

In [ ]:
options_data.info()

In [ ]:
# Visualize distributions of numerical features
numerical_cols = ['strike_price', 'best_bid', 'best_offer', 'volume', 'open_interest', 'impl_volatility', 'delta', 'gamma', 'vega', 'theta', 'contract_size']
options_data[numerical_cols].hist(figsize=(15, 10))
plt.suptitle("Histograms of Numerical Features")
plt.show()


In [ ]:

# # Visualize relationships between features
# sns.pairplot(options_data[numerical_cols])
# plt.suptitle("Pairplot of Numerical Features")
# plt.show()


In [ ]:

# Visualize relationships with target variable (if available)
sns.boxplot(x=options_data['cp_flag'], y=options_data['best_bid'])
plt.title("Boxplot of Best Bid by Call/Put Flag")
plt.show()

### Download the corresponding stock price

In [48]:
tickers = options_data['ticker'].unique()

print(tickers)

# Download historical stock data of the correspoonding ticker
stock_data = {}
for ticker in tickers:
    start_date = options_data['date'].min()
    end_date = (pd.to_datetime(options_data['date'].max()) + pd.Timedelta(days=1)).date()

    stock_data[ticker] = yf.download(ticker, start=start_date, end=end_date)
    stock_data[ticker]['ticker'] = ticker  # Add a new column 'Ticker' with ticker symbol


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed

['AMD' 'NVDA' 'TSM' 'SMCI']


In [49]:
# Combine all stock data into a single DataFrame
merged_stock_data = pd.concat(stock_data.values())
merged_stock_data['date'] = merged_stock_data.index
merged_stock_data.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1008 entries, 2022-02-28 to 2023-02-28
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Open       1008 non-null   float64       
 1   High       1008 non-null   float64       
 2   Low        1008 non-null   float64       
 3   Close      1008 non-null   float64       
 4   Adj Close  1008 non-null   float64       
 5   Volume     1008 non-null   int64         
 6   ticker     1008 non-null   object        
 7   date       1008 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 70.9+ KB


In [42]:
merged_stock_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1008 entries, 2022-02-28 to 2023-02-28
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1008 non-null   float64
 1   High       1008 non-null   float64
 2   Low        1008 non-null   float64
 3   Close      1008 non-null   float64
 4   Adj Close  1008 non-null   float64
 5   Volume     1008 non-null   int64  
 6   Ticker     1008 non-null   object 
dtypes: float64(5), int64(1), object(1)
memory usage: 63.0+ KB


In [44]:
merged_stock_data.head()

In [50]:
merged_stock_data['ticker'].unique()

array(['AMD', 'NVDA', 'TSM', 'SMCI'], dtype=object)

In [51]:
# Merge options data and stock data based on common columns (ticker and date)
combined_data = pd.merge(options_data, merged_stock_data, on=['ticker', 'date'], how='left')

# Print or further process the combined data
# print(combined_data)

              date     exdate cp_flag  strike_price  best_bid  best_offer  \
0       2022-02-28 2022-03-04       P        143000     18.30       21.70   
1       2022-02-28 2022-03-04       P        144000     19.65       22.55   
2       2022-02-28 2022-03-04       P        145000     20.45       23.70   
3       2022-02-28 2022-03-04       P        146000     21.30       24.75   
4       2022-02-28 2022-03-04       P        147000     23.50       25.75   
...            ...        ...     ...           ...       ...         ...   
1166639 2023-02-28 2023-12-15       P         75000      8.10        9.10   
1166640 2023-02-28 2023-12-15       P         80000      9.80       10.90   
1166641 2023-02-28 2023-12-15       P         85000     11.80       12.60   
1166642 2023-02-28 2023-12-15       P         90000     13.90       14.60   
1166643 2023-02-28 2023-12-15       P         95000     16.30       16.80   

         volume  open_interest  impl_volatility     delta     gamma  \
0   

In [ ]:
#  0   date             1166644 non-null  datetime64[ns]
#  1   exdate           1166644 non-null  datetime64[ns]
#  2   cp_flag          1166644 non-null  object        
#  3   strike_price     1166644 non-null  int64         
#  4   best_bid         1166644 non-null  float64       
#  5   best_offer       1166644 non-null  float64       
#  6   volume           1166644 non-null  int64         
#  7   open_interest    1166644 non-null  int64         
#  8   impl_volatility  1166644 non-null  float64       
#  9   delta            1166644 non-null  float64       
#  10  gamma            1166644 non-null  float64       
#  11  vega             1166644 non-null  float64       
#  12  theta            1166644 non-null  float64       
#  13  contract_size    1166644 non-null  int64   

In [52]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1166644 entries, 0 to 1166643
Data columns (total 21 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   date             1166644 non-null  datetime64[ns]
 1   exdate           1166644 non-null  datetime64[ns]
 2   cp_flag          1166644 non-null  object        
 3   strike_price     1166644 non-null  int64         
 4   best_bid         1166644 non-null  float64       
 5   best_offer       1166644 non-null  float64       
 6   volume           1166644 non-null  int64         
 7   open_interest    1166644 non-null  int64         
 8   impl_volatility  1166644 non-null  float64       
 9   delta            1166644 non-null  float64       
 10  gamma            1166644 non-null  float64       
 11  vega             1166644 non-null  float64       
 12  theta            1166644 non-null  float64       
 13  contract_size    1166644 non-null  int64         
 14  ti

## Exploratory Data Analysis

In [ ]:

# Step 3: Feature Engineering, compute days to expiration
data['dte'] = (data['exdate'] - data['date']).dt.days
data['moneyness'] = data['Adj Close'] / data['strike_price']

